In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data=pd.read_csv("CKD.csv")

In [3]:
data=pd.get_dummies(data,dtype=int,drop_first=True)

In [4]:
data["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [5]:
data.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [6]:
indep=data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv','wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal','pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes','appet_yes', 'pe_yes', 'ane_yes']]

In [7]:
data.shape

(399, 28)

In [8]:
dep=data[['classification_yes']]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [12]:
param_grid={'criterion':["gini", "entropy", "log_loss"],'splitter':["best", "random"],'max_features':["sqrt", "log2"],'max_leaf_nodes':[2,4,6,8,10]}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)
re=grid.cv_results_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [13]:
y_pred=grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred,y_test)

In [15]:
from sklearn.metrics import classification_report
clf=classification_report(y_pred,y_test)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_pred,y_test)
f1_macro

0.975609756097561

In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9833811573409852

In [18]:
print("The f1_macro value for best parameter():",format(grid.best_params_),f1_macro)
table=pd.DataFrame.from_dict(re)
table
print(clf)

The f1_macro value for best parameter(): {'criterion': 'entropy', 'max_features': 'log2', 'max_leaf_nodes': 8, 'splitter': 'best'} 0.975609756097561
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        51
           1       0.98      0.98      0.98        82

    accuracy                           0.97       133
   macro avg       0.97      0.97      0.97       133
weighted avg       0.97      0.97      0.97       133

